# Impmort

In [23]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import StandardScaler,MinMaxScaler

# Read data

In [24]:
data_dir = 'data'
af = 'u'

pf1 = f'7200-3600'
pf = f'7200-3600.__2__'

df__train = pd.read_csv(f'{data_dir}/tr.{af}4.{pf1}.csv', index_col=[0])
df__test = pd.read_csv(f'{data_dir}/t.{af}4.{pf1}.csv', index_col=[0])
# df__val = pd.read_csv(f'{data_dir}/v.{af}4.{pf1}.csv', index_col=[0])

In [25]:
print(len(df__train), len(df__test))

279418 18226


In [26]:
# df__test['Label']

In [27]:
df__test.loc[df__test['Label'] == 5].head()

,Conversation,SrcAddr,DstAddr,State,Proto,n_flows,StreamID_unique,Sport_nunique,Sport_mean,Sport_std,...,DstBytesPerSec_median,DstBytesPerSec_RU,BytesPerPkt_mean,BytesPerPkt_std,BytesPerPkt_max,BytesPerPkt_median,BytesPerPkt_RU,Label,window_id,LabelStr
5306,41.232.73.23 -> 150.35.87.168,41.232.73.23,150.35.87.168,alltcp,tcp,2,[ 1976 51591],2,6668.0,1.414214,...,1.725578,1.0,81.047175,10.549573,88.506849,81.047175,1.0,5,0,flow=From-Botnet-V44-TCP-CC107-IRC-Not-Encrypted
3458,41.232.73.23 -> 150.35.87.168,41.232.73.23,150.35.87.168,alltcp,tcp,2,[51591 84191],2,6666.5,0.707107,...,1.735833,1.0,81.148486,10.406297,88.506849,81.148486,1.0,5,1,flow=From-Botnet-V44-TCP-CC107-IRC-Not-Encrypted
1934,41.232.73.23 -> 150.35.87.168,41.232.73.23,150.35.87.168,alltcp,tcp,2,[ 84191 101058],2,6667.0,1.414214,...,0.609414,1.0,73.265432,0.742026,73.790123,73.265432,1.0,5,2,flow=From-Botnet-V44-TCP-CC107-IRC-Not-Encrypted
637,41.232.73.23 -> 150.35.87.168,41.232.73.23,150.35.87.168,alltcp,tcp,2,[101058 109906],2,6667.5,0.707107,...,0.593343,1.0,73.045370,0.430811,73.350000,73.045370,1.0,5,3,flow=From-Botnet-V44-TCP-CC107-IRC-Not-Encrypted
165,41.232.73.23 -> 150.35.87.168,41.232.73.23,150.35.87.168,alltcp,tcp,2,[109906 115675],2,6667.5,0.707107,...,0.587141,1.0,73.212500,0.194454,73.350000,73.212500,1.0,5,4,flow=From-Botnet-V44-TCP-CC107-IRC-Not-Encrypted


In [28]:
"','".join(df__train.columns)

"Conversation','SrcAddr','DstAddr','State','Proto','n_flows','StreamID_unique','Sport_nunique','Sport_mean','Sport_std','Sport_max','Sport_median','Sport_RU','Dport_nunique','Dport_mean','Dport_std','Dport_max','Dport_median','Dport_RU','TotBytes_sum','TotBytes_mean','TotBytes_std','TotBytes_max','TotBytes_median','TotBytes_RU','SrcBytes_sum','SrcBytes_mean','SrcBytes_std','SrcBytes_max','SrcBytes_median','SrcBytes_RU','DstBytes_sum','DstBytes_mean','DstBytes_std','DstBytes_max','DstBytes_median','DstBytes_RU','TotPkts_sum','TotPkts_mean','TotPkts_std','TotPkts_max','TotPkts_median','TotPkts_RU','PktsPerSec_mean','PktsPerSec_std','PktsPerSec_max','PktsPerSec_median','PktsPerSec_RU','BytesPerSec_mean','BytesPerSec_std','BytesPerSec_max','BytesPerSec_median','BytesPerSec_RU','SrcBytesPerSec_mean','SrcBytesPerSec_std','SrcBytesPerSec_max','SrcBytesPerSec_median','SrcBytesPerSec_RU','DstBytesPerSec_mean','DstBytesPerSec_std','DstBytesPerSec_max','DstBytesPerSec_median','DstBytesPerSec_RU',

# One-hot encode

## Common Proto & State values

In [29]:
df__train.loc[(df__train['State'].str.contains('_')), 'State'] = 'alltcp'
df__test.loc[(df__test['State'].str.contains('_')), 'State'] = 'alltcp'
print(df__train['Proto'].value_counts(), '\n')
print(df__train['State'].value_counts())

udp     185550
tcp      86110
icmp      5177
arp       1878
igmp       640
rtp         29
ipv6        17
rtcp        15
rsvp         2
Name: Proto, dtype: int64 

CON         176115
alltcp       86110
INT          10309
ECR           1287
REQ           1257
RED           1245
ECO            726
URP            689
URH            455
RSP            450
TXD            330
URHPRO         221
URN            183
URO             17
URFIL           16
DCE              5
URF              2
nanvalue         1
Name: State, dtype: int64


In [30]:
df__train['Proto'].value_counts()[:3]

udp     185550
tcp      86110
icmp      5177
Name: Proto, dtype: int64

## Encode

In [31]:
""" replace infrequent state with other """
# fr = df__train['State'].value_counts()
# filt = fr[fr > 100]
# common_state_list = list(filt.index)
common_state_list = list(df__train['State'].value_counts()[:20].index)
print(common_state_list)
df__train.loc[(~df__train['State'].str.contains('_')) & (~df__train['State'].isin(common_state_list)), 'State'] = 'other'
df__test.loc[(~df__test['State'].str.contains('_')) & (~df__test['State'].isin(common_state_list)), 'State'] = 'other'
# df__val.loc[(~df__val['State'].str.contains('_')) & (~df__val['State'].isin(common_state_list)), 'State'] = 'other'

['CON', 'alltcp', 'INT', 'ECR', 'REQ', 'RED', 'ECO', 'URP', 'URH', 'RSP', 'TXD', 'URHPRO', 'URN', 'URO', 'URFIL', 'DCE', 'URF', 'nanvalue']


In [32]:
""" replace infrequent proto with other """
# fr = df__train['Proto'].value_counts()
# filt = fr[fr > 100]
# common_proto_list = list(filt.index)
common_proto_list = list(df__train['Proto'].value_counts()[:3].index)
print(common_proto_list)
df__train.loc[~df__train['Proto'].isin(common_proto_list), 'Proto'] = 'other'
df__test.loc[~df__test['Proto'].isin(common_proto_list), 'Proto'] = 'other'
# df__val.loc[~df__val['Proto'].isin(common_proto_list), 'Proto'] = 'other'

['udp', 'tcp', 'icmp']


In [33]:
# df__train['State'].value_counts()

In [34]:
""" 
onehot encode string fields 
"""

from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder


""" get categories to encode """
proto_list = common_proto_list + ['other']
state_list = common_state_list + ['other']
categories = [proto_list, state_list]

onehot_cols_proto = ['P_'+p for p in proto_list]
onehot_cols_state = ['S_'+p for p in state_list]
onehot_cols = onehot_cols_proto + onehot_cols_state

print('proto_list', proto_list)
print('state_list', state_list)
print('categories', categories)
print('onehot_cols', onehot_cols)

""" encode. fit & transform """
# ct_train = make_column_transformer((OneHotEncoder(categories='auto', handle_unknown='infrequent_if_exist', max_categories=15), [0, 1]), remainder="passthrough")
ct_train = make_column_transformer((OneHotEncoder(categories=categories), [0,1]), remainder="passthrough")
Xtrain_enc = ct_train.fit_transform(df__train[['Proto','State']])
Xtrain_enc = Xtrain_enc.astype('float64').toarray()
df_Xtrain_enc = pd.DataFrame(Xtrain_enc, columns=onehot_cols)

Xtest_enc = ct_train.transform(df__test[['Proto','State']])
Xtest_enc = Xtest_enc.astype('float64').toarray()
df_Xtest_enc = pd.DataFrame(Xtest_enc, columns=onehot_cols)

# Xval_enc = ct_train.transform(df__val[['Proto','State']])
# Xval_enc = Xval_enc.astype('float64').toarray()
# df_Xval_enc = pd.DataFrame(Xval_enc, columns=onehot_cols)

proto_list ['udp', 'tcp', 'icmp', 'other']
state_list ['CON', 'alltcp', 'INT', 'ECR', 'REQ', 'RED', 'ECO', 'URP', 'URH', 'RSP', 'TXD', 'URHPRO', 'URN', 'URO', 'URFIL', 'DCE', 'URF', 'nanvalue', 'other']
categories [['udp', 'tcp', 'icmp', 'other'], ['CON', 'alltcp', 'INT', 'ECR', 'REQ', 'RED', 'ECO', 'URP', 'URH', 'RSP', 'TXD', 'URHPRO', 'URN', 'URO', 'URFIL', 'DCE', 'URF', 'nanvalue', 'other']]
onehot_cols ['P_udp', 'P_tcp', 'P_icmp', 'P_other', 'S_CON', 'S_alltcp', 'S_INT', 'S_ECR', 'S_REQ', 'S_RED', 'S_ECO', 'S_URP', 'S_URH', 'S_RSP', 'S_TXD', 'S_URHPRO', 'S_URN', 'S_URO', 'S_URFIL', 'S_DCE', 'S_URF', 'S_nanvalue', 'S_other']


In [35]:
df__train = pd.concat([df__train.reset_index(), df_Xtrain_enc], axis=1).drop(columns=['index'])
df__test = pd.concat([df__test.reset_index(), df_Xtest_enc], axis=1).drop(columns=['index'])
# df__val = pd.concat([df__val.reset_index(), df_Xval_enc], axis=1).drop(columns=['index'])

df__test.head()

,Conversation,SrcAddr,DstAddr,State,Proto,n_flows,StreamID_unique,Sport_nunique,Sport_mean,Sport_std,...,S_RSP,S_TXD,S_URHPRO,S_URN,S_URO,S_URFIL,S_DCE,S_URF,S_nanvalue,S_other
0,112.68.196.144 -> 150.35.88.126,112.68.196.144,150.35.88.126,INT,udp,3,[ 912 4512 53830],2,57821.666667,0.577350,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,112.68.196.144 -> 150.35.88.126,112.68.196.144,150.35.88.126,alltcp,tcp,4,[36533 48509 59196 70068],4,2319.750000,1202.688204,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,112.68.196.144 -> 150.35.88.126,112.68.196.144,150.35.88.126,alltcp,tcp,6,[ 2243 6571 8988 12981 21172 25862],6,2499.500000,1049.083552,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,150.35.87.141 -> 150.35.83.12,150.35.87.141,150.35.83.12,CON,udp,17010,[ 20 21 22 ... 82178 82179 82180],12788,46925.880658,8154.914736,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,150.35.87.141 -> 150.35.83.16,150.35.87.141,150.35.83.16,URP,icmp,2,[ 472 49121],1,771.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Standardizing
    
As we have seen that the range of few features in this dataset is very large. So we will keep everything within certain range by applying standardscaler. After this all the features will have mean 0 and std 1

In [36]:
# Standardizing the data
numeric_cols = [
       'n_flows',
       # 'Sport_nunique','Sport_std','Sport_median','Sport_RU','Dport_nunique','Dport_mean','Dport_std','Dport_max','Dport_median','Dport_RU','TotBytes_sum','TotBytes_mean','TotBytes_std','TotBytes_max','TotBytes_median','TotBytes_RU','SrcBytes_sum','SrcBytes_mean','SrcBytes_std','SrcBytes_max','SrcBytes_median','SrcBytes_RU','DstBytes_sum','DstBytes_mean','DstBytes_std','DstBytes_max','DstBytes_median','DstBytes_RU','TotPkts_sum','TotPkts_mean','TotPkts_std','TotPkts_max','TotPkts_median','TotPkts_RU','PktsPerSec_std','PktsPerSec_median','PktsPerSec_RU','BytesPerSec_std','BytesPerSec_median','BytesPerSec_RU','SrcBytesPerSec_mean','SrcBytesPerSec_std','SrcBytesPerSec_median','SrcBytesPerSec_RU','DstBytesPerSec_mean','DstBytesPerSec_std','DstBytesPerSec_max','DstBytesPerSec_median','DstBytesPerSec_RU','BytesPerPkt_std','BytesPerPkt_median','BytesPerPkt_RU','PktsPerSec_mean','PktsPerSec_max','BytesPerSec_mean','BytesPerSec_max','SrcBytesPerSec_max','','BytesPerPkt_max',
       'Sport_mean','Sport_max',
       'BytesPerPkt_mean','PktsPerSec_mean','BytesPerSec_mean','BytesPerPkt_max','BytesPerSec_max','SrcBytesPerSec_max',
]

In [37]:
ms = MinMaxScaler()
ms = ms.fit(df__train[numeric_cols].values)
df__train[numeric_cols] = ms.transform(df__train[numeric_cols].values)
df__test[numeric_cols] = ms.transform(df__test[numeric_cols].values)
# df__val[numeric_cols] = ss.transform(df__val[numeric_cols].values)

# print(ss.min_)
# print(ss.scale_)

# X y

In [38]:
fts_cols = numeric_cols + onehot_cols

In [39]:
#? creating x and y set from the dataset
X_train, y_train = df__train[fts_cols].values, df__train['Label'].values
X_test, y_test = df__test[fts_cols].values, df__test['Label'].values
# X_val, y_val = df__val[fts_cols].values, df__val['Label'].values

In [40]:
#? for binary classification
y_train[y_train != 5] = 0
y_train[y_train == 5] = 1

y_test[y_test != 5] = 0
y_test[y_test == 5] = 1

# y_val[y_val != 5] = 0
# y_val[y_val == 5] = 1

print(X_train.shape, y_train.shape)

(279418, 32) (279418,)


# Save ?

In [41]:
np.save(f'data/{af}4-3.tr.X.{pf}.npy', X_train)
np.save(f'data/{af}4-3.tr.y.{pf}.npy', y_train)

np.save(f'data/{af}4-3.t.X.{pf}.npy', X_test)
np.save(f'data/{af}4-3.t.y.{pf}.npy', y_test)

# np.save(f'data/{af}4-3.v.X.{pf}.npy', X_val)
# np.save(f'data/{af}4-3.v.y.{pf}.npy', y_val)

In [42]:
open(f'data/{af}4-3.fts_cols.{pf}.txt', 'w').write('\n'.join(fts_cols))

285

In [46]:
""" save the encoder """
import pickle

pickle.dump(ct_train, open(f'data/{af}4-3.ohe.{pf}.pkl', 'wb'))
pickle.dump(ms, open(f'data/{af}4-3.mms.{pf}.pkl', 'wb'))

In [44]:
"','".join(fts_cols)

"n_flows','Sport_mean','Sport_max','BytesPerPkt_mean','PktsPerSec_mean','BytesPerSec_mean','BytesPerPkt_max','BytesPerSec_max','SrcBytesPerSec_max','P_udp','P_tcp','P_icmp','P_other','S_CON','S_alltcp','S_INT','S_ECR','S_REQ','S_RED','S_ECO','S_URP','S_URH','S_RSP','S_TXD','S_URHPRO','S_URN','S_URO','S_URFIL','S_DCE','S_URF','S_nanvalue','S_other"

In [45]:
#? cleanup
del X_train
del y_train
del df__train

del X_test
del y_test
del df__test

# del X_val
# del y_val
# del df__val